# Part 1: Launch a Duet Server and upload data

In [1]:
import syft as sy
duet = sy.launch_duet()

🎤  🎸  ♪♪♪ Starting Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!
♫♫♫ > Duet Server ID: c428f674d926eb340a98d6ce00d07bfd

♫♫♫ > STEP 1: Send the following code to your Duet Partner!

import syft as sy
duet = sy.duet("c428f674d926eb340a98d6ce00d07bfd")

♫♫♫ > STEP 2: Ask your partner for their Client ID and enter it below!

♫♫♫ > Connecting...


C:\Users\dylan\anaconda3\envs\new_pysyft\lib\site-packages\aiortc\rtcdtlstransport.py:211: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  _openssl_assert(lib.SSL_CTX_use_certificate(ctx, self._cert._x509) == 1)  # type: ignore
C:\Users\dylan\anaconda3\envs\new_pysyft\lib\site-packages\aiortc\rtcdtlstransport.py:186: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  value=certificate_digest(self._cert._x509),  # type: ignore



♫♫♫ > CONNECTED!

`searchable` is deprecated please use `pointable` in futurees: 0  Request Handlers: 0                                
`searchable` is deprecated please use `pointable` in futurees: 1  Request Handlers: 0                                
`searchable` is deprecated please use `pointable` in futurees: 4  Request Handlers: 1                                


In [2]:
import torch as th
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd

Read in data

In [3]:
df_X_train_1 = pd.read_csv('ETL/X_train_1.csv')
df_X_val_1 = pd.read_csv('ETL/X_val_1.csv')
df_y_train_1 = pd.read_csv('ETL/y_train_1.csv')
df_y_val_1 = pd.read_csv('ETL/y_val_1.csv')

Conver to NumPy array

In [4]:
X_train_1_np = df_X_train_1.to_numpy()
X_val_1_np = df_X_val_1.to_numpy()
y_train_1_np = df_y_train_1.to_numpy()
y_val_1_np = df_y_val_1.to_numpy()

Convert to Torch tensor

In [5]:
X_train_1 = th.from_numpy(X_train_1_np).float()
X_val_1 = th.from_numpy(X_val_1_np).float()
y_train_1 = th.from_numpy(y_train_1_np).float()
y_val_1 = th.from_numpy(y_val_1_np).float()

In [6]:
#data = th.FloatTensor(np.array([5, 15, 25, 35, 45, 55]).reshape(-1, 1))

X_train_1 = X_train_1.tag("DO2 training X data")
X_train_1 = X_train_1.describe("number of samples and features")

data_ptr = X_train_1.send(duet, searchable=True)

In [10]:
y_train_1 = y_train_1.tag("DO2 training y data")
y_train_1 = y_train_1.describe("number of samples and features")

target_ptr = y_train_1.send(duet, searchable=True)

In [11]:
duet.store.pandas

,ID,Tags,Description,object_type
0,<UID: 9d473476b9334f4bad413cca4a823e54>,[DO2 training X data],number of samples and features,<class 'torch.Tensor'>
1,<UID: 098db24261474e639bf592added98ba1>,[DO1 training y data],number of samples and features,<class 'torch.Tensor'>
2,<UID: cdc9f879951b461b94b9329afc830296>,[DO2 training y data],number of samples and features,<class 'torch.Tensor'>


[2022-01-04T19:57:53.078376+0000][CRITICAL][logger]][61380] > HANDLER Request <UID: 39f507880580409ab7a87d55aea7710e>: To evaluate training progress
Value: <Storable: 0.9064223766326904>
[2022-01-04T19:57:53.294801+0000][CRITICAL][logger]][61380] > HANDLER Request <UID: e12d5c8e61d441dc8f1c74f2bcd6359a>: To evaluate training progress
Value: <Storable: 0.24471142888069153>
[2022-01-04T19:57:53.531166+0000][CRITICAL][logger]][61380] > HANDLER Request <UID: 312a39df49c7451abbc1aeb909e11c49>: To evaluate training progress
Value: <Storable: 0.2971583604812622>
[2022-01-04T19:57:53.748586+0000][CRITICAL][logger]][61380] > HANDLER Request <UID: 12c40f77320a43a5868dabb641479661>: To evaluate training progress
Value: <Storable: 0.3697357177734375>
[2022-01-04T19:57:53.966006+0000][CRITICAL][logger]][61380] > HANDLER Request <UID: a96c96a03ea24f8d8c54ed293c722f95>: To evaluate training progress
Value: <Storable: 0.3653009533882141>
[2022-01-04T19:57:54.183425+0000][CRITICAL][logger]][61380] > HA

In [9]:
duet.requests.add_handler(
    action="accept",
    print_local=True,  # print the result in your notebook
)